In [ ]:
#process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

In [1]:
import sys
import json
import re
import collections
import os
import pandas as pd
import numpy as np
#import datetime
import csv
import scipy.sparse as sp
import scipy.sparse.linalg  as la
import itertools

### Number of tracks depending on constraints
iterates over the million playlist dataset and outputs number songs tha appear more than: 1,2,3,4 times

In [ ]:
total_playlists = 0
total_tracks = 0
track_histogram = collections.Counter()

quick = False
max_files_for_quick_processing = 1

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            process_info(mpd_slice['info'])
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break

    show_summary()


def show_summary():
    print()
    
 
    t1 = len(list(filter(lambda y: y[1]>1, track_histogram.items())))
    t2 = len(list(filter(lambda y: y[1]>2, track_histogram.items())))
    t3 = len(list(filter(lambda y: y[1]>3, track_histogram.items())))
    t4 = len(list(filter(lambda y: y[1]>4, track_histogram.items())))
    t5 = len(list(filter(lambda y: y[1]>5, track_histogram.items())))

    unique = len(track_histogram)
    
    print ("MPD data percentage computed: "+ str(total_playlists/1000000))
    print ("number of playlists", total_playlists)
    print ("number of tracks", total_tracks)
    print ("number of unique tracks appearing ate least once   :" + str(unique) + " that is 100%")
    print ("number of unique tracks appearing more than 1 time :" + str(t1) + " that is "+ str(round(t1/(unique)*100,2))+"%" )
    print ("number of unique tracks appearing more than 2 times:"+ str(t2) + " that is "+ str(round(t2/(unique)*100,2))+"%" )
    print ("number of unique tracks appearing more than 3 times:"+ str(t3) + " that is "+ str(round(t3/(unique)*100,2))+"%"  )
    print ("number of unique tracks appearing more than 4 times:"+ str(t4) + " that is "+ str(round(t4/(unique)*100,2))+"%"  )
    print ("number of unique tracks appearing more than 5 times:"+ str(t5) + " that is "+ str(round(t5/(unique)*100,2))+"%" )


def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name


def process_playlist(playlist):
    global total_tracks, total_playlists
    total_playlists += 1
    for track in playlist['tracks']:
        total_tracks += 1
        track_histogram[track['track_uri']] += 1


def process_info(_):
    pass


if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = False
    #process_mpd(path)
    process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

In [ ]:
"""
>1 100%
>2 44%
>3 30%
>4 20%
>5 ...
"""

### CSV file maker      _track uri - numeric ID_
Makes a csv containing only the tracks appearing more than "num" times in the database, and gives each track_uri a numeric ID

In [ ]:
"""
in this notebook we will create a csv file with all the tracks_uris and its new associated unique numeric ID
only with tracks appearing more than num = 4 times 
data format: id track_uri
usage: process_mpd(path)
"""
total_tracks = 0
track_histogram = collections.Counter()
num = 4

quick = False
max_files_for_quick_processing = 1

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break

    show_summary()
    write_file()

def write_file():
    j = 0
    with open('id_trackuri_filter.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(["id","track_uri"])
        for track in track_histogram.items():
            if track[1]>num:
                spamwriter.writerow([j,track[0][14:]])
                j+=1
        

def show_summary():
    t1 = len(list(filter(lambda y: y[1]>4, track_histogram.items())))
    print("number of tracks", total_tracks)
    print("number of unique tracks", len(track_histogram))
    print("saved tracks in file, with num_appearences>"+str(num)+" times: "+ str(t1) +
          " that is "+ str(round(t1/len(track_histogram)*100,2))+"%" )


def process_playlist(playlist):
    global total_tracks
    for track in playlist['tracks']:
        total_tracks += 1
        track_histogram[track['track_uri']] += 1

if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = False
    #process_mpd(path)
    process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

## Clean data slices with playists by id
creates a new set of data slices with less information than the original one
specifically, for each slice, it will create a csv file containing its 1000 playlists, with only the
tracks tha apear more than "num" times in the whole dataset, each track written as a numeric ID 

In [2]:
quick = False
max_files_for_quick_processing = 2
#csv to dict
reader = csv.reader(open('id_trackuri_filter.csv', 'r'))
dict_trackuri_ids = {v:k for (k,v) in reader}

def process_mpd(path):
    count = 0 # slices counter
    i=0       # playlists counter   
    filenames = os.listdir(path)
    
    #for each slice
    for filename in sorted(filenames):
        #read slice
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)

            #process slice and write a csv 
            with open('../MPD_filter_ids/playlists_tracks_id_'+str(count)+'.csv', 'w', newline='',encoding="utf-8") as csvfile:
                spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
                spamwriter.writerow(["playlist","track_id"])
                for playlist in mpd_slice['playlists']:
                    for track in playlist["tracks"]:
                        uri = track["track_uri"][14:]
                        if uri in dict_trackuri_ids.keys():
                            spamwriter.writerow([i, dict_trackuri_ids[uri]])
                    i+=1
        count += 1

        if quick and count > max_files_for_quick_processing:
            break


if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = False
    #process_mpd(path)
    process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

## Sparse Matrix
Makes a matrix tracks x tracks with the number of occurences in the same playlist

In [ ]:
"""
This notebooks creates and saves into a file sparse_matrix_filter.npz a sparse matrix tracks x tracks
containing the total amount of times 2 tracks appear in the same playlist
usage: run after running FILE CREATOR - playlist by id, so that the file playlists_tracks_id.csv is in the same folder
"""
quick = True
max_files_for_quick_processing = 2
size_matrix = 133500 #number of unique songs in 10% MPD filtered 
matrix = sp.coo_matrix((size_matrix, size_matrix), [np.int64])

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("playlists_tracks_id_") and filename.endswith(".csv"):
            fullpath = os.sep.join((path, filename))
            process_slice(fullpath)
            #df = pd.read_csv(fullpath)
            #process_slice(df)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
        sp.save_npz('sparse_matrix_filter.npz', matrix)


def process_slice(fullpath):
    df = pd.read_csv(fullpath)
    i = 0 # playlist iterator
    j = 0 # row from csv iterator
    length_csv = len(df)
    play = []
    global matrix
    
    while j<length_csv:
        if df["playlist"][j]==i:
            play.append(df["track_id"][j])
            j+=1
        else:
            #we make a sparse matrix out of the ith playlist
            combinations = list(itertools.product(play,play)) #TODO: CONSIDER PAIRS ONLY ONCE!!!
            row = np.array([m for (m,n) in combinations])
            col = np.array([n for (m,n) in combinations])
            n = (len(row))
            listones = list(np.ones(len(row), dtype=np.int64))
            mat = sp.csc_matrix((listones, (row, col)), shape=(size_matrix,size_matrix))

            #adds up the matrix to the final sparse matrix
            matrix +=mat

            #cleans the list for the next playlist
            play = []
            i+=1


if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = True
    #process_mpd(path)
    process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD_filter_ids")






In [ ]:
sp.save_npz('sparse_matrix_filter.npz', matrix)

In [5]:
df = pd.read_csv("../MPD_filter_ids/playlists_tracks_id_0.csv")

In [8]:
df["playlist"][1]

0

In [9]:
len(df)

60235

In [ ]:
print(matrix[378843,172352])
print(matrix[75800,65296])
print(matrix[267752,43983])
print(matrix[33101,33101])
print(matrix.max())